In [55]:
import sqlalchemy as sql
import pandas as pd
import numpy as np
import csv
import json
from jsonpath_ng.ext import parse 
import re
from pprint import pprint


# Stanza
import stanza

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
from spacy import displacy


# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [34]:
awards_filepath = "src/files/220k_awards_by_directors.csv"
movies_filepath     = "src/files/AllMoviesDetailsCleaned.csv"
cast_filepath       = "src/files/AllMoviesCastingRaw.csv"

awards_df = pd.read_csv(awards_filepath, delimiter=",", na_values='none')
movies_df = pd.read_csv(movies_filepath, delimiter=";", na_values='none')
cast_df = pd.read_csv(cast_filepath, delimiter=";", na_values='none')




/tmp/ipykernel_76384/4140812894.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv(movies_filepath, delimiter=";", na_values='none')


In [35]:
awards_df

,director_name,ceremony,year,category,outcome,original_language
0,Aki Kaurismäki,ACCEC Awards,2004.0,ACCEC Award,Won,fi
1,Aki Kaurismäki,Alliance of Women Film Journalists,2012.0,EDA Award,Nominated,fi
2,Aki Kaurismäki,"Amanda Awards, Norway",1999.0,Amanda,Nominated,fi
3,Aki Kaurismäki,Argentinean Film Critics Association Awards,2013.0,Silver Condor,Nominated,fi
4,Aki Kaurismäki,Argentinean Film Critics Association Awards,2004.0,Silver Condor,Nominated,fi
...,...,...,...,...,...,...
225670,Courtney Hoffman,Satellite Awards,2016.0,Satellite Award,Nominated,en
225671,Fanni Metelius,Berlin International Film Festival,2012.0,Crystal Bear,Nominated,sv
225672,Fanni Metelius,Guldbagge Awards,2015.0,Guldbagge,Nominated,sv
225673,Fanni Metelius,Uppsala International Short Film Festival,2012.0,Honorable Mention,Won,sv


In [36]:
movies_df.head()

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,production_companies_number,production_countries_number,spoken_languages_number
0,2,0,Drama|Crime,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,0.823904,Villealfa Filmproduction Oy,Finland,...,69.0,suomi,Released,NaN,Ariel,7.1,40,2,1,2
1,3,0,Drama|Comedy,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",0.47445,Villealfa Filmproduction Oy,Finland,...,76.0,English,Released,NaN,Shadows in Paradise,7.0,32,1,1,3
2,5,4000000,Crime|Comedy,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,1.698,Miramax Films,United States of America,...,98.0,English,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,485,2,1,1
3,6,0,Action|Thriller|Crime,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",1.32287,Universal Pictures,Japan,...,110.0,English,Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,6.5,69,3,2,1
4,8,42000,Documentary,tt0825671,en,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,0.054716,inLoops,Austria,...,80.0,English,Released,A Megacities remix.,Life in Loops (A Megacities RMX),6.4,4,1,1,5


In [37]:
movies_filtered_df = movies_df.head(30)

In [38]:
movies_filtered_df

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,production_companies_number,production_countries_number,spoken_languages_number
0,2,0,Drama|Crime,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,0.823904,Villealfa Filmproduction Oy,Finland,...,69.0,suomi,Released,NaN,Ariel,7.1,40,2,1,2
1,3,0,Drama|Comedy,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",0.47445,Villealfa Filmproduction Oy,Finland,...,76.0,English,Released,NaN,Shadows in Paradise,7.0,32,1,1,3
2,5,4000000,Crime|Comedy,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,1.698,Miramax Films,United States of America,...,98.0,English,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,485,2,1,1
3,6,0,Action|Thriller|Crime,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",1.32287,Universal Pictures,Japan,...,110.0,English,Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,6.5,69,3,2,1
4,8,42000,Documentary,tt0825671,en,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,0.054716,inLoops,Austria,...,80.0,English,Released,A Megacities remix.,Life in Loops (A Megacities RMX),6.4,4,1,1,5
5,9,0,Drama,tt0425473,de,Sonntag im August,NaN,0.001647,NaN,Germany,...,15.0,Deutsch,Released,NaN,Sunday in August,5.3,2,0,1,1
6,11,11000000,Adventure|Action|Science Fiction,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,10.492614,Lucasfilm,United States of America,...,121.0,English,Released,"A long time ago in a galaxy far, far away...",Star Wars,8.0,6168,2,1,1
7,12,94000000,Animation|Family,tt0266543,en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",9.915573,Pixar Animation Studios,United States of America,...,100.0,English,Released,"There are 3.7 trillion fish in the ocean, they...",Finding Nemo,7.6,5531,1,1,1
8,13,55000000,Comedy|Drama|Romance,tt0109830,en,Forrest Gump,A man with a low IQ has accomplished great thi...,10.351236,Paramount Pictures,United States of America,...,142.0,English,Released,"The world will never be the same, once you've ...",Forrest Gump,8.2,7204,1,1,1
9,14,15000000,Drama,tt0169547,en,American Beauty,"Lester Burnham, a depressed suburban father in...",8.191009,DreamWorks SKG,United States of America,...,122.0,English,Released,Look closer.,American Beauty,7.9,2994,2,1,1


In [39]:
# Convert to list
data = movies_filtered_df.overview.values.tolist()

# Filters
# Remove speical characters - except for , and -
data = [re.sub('[^a-zA-Z0-9,@\- ]', '', str(sent)) for sent in data]

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

# Remove empty strings and blank spaces

data = [ x for x in data if (len(x.strip()) > 0)]

# Convert everything to lower case

data = [ str(x).lower() for x in data]




In [40]:
data[1]

'an episode in the life of nikander, a garbage man, involving the death of a co-worker, an affair and much more'

## Convert sentence to words

In [41]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[0])

['taisto', 'kasurinen', 'is', 'finnish', 'coal', 'miner', 'whose', 'father', 'has', 'just', 'committed', 'suicide', 'and', 'who', 'is', 'framed', 'for', 'crime', 'he', 'did', 'not', 'commit', 'in', 'jail', 'he', 'starts', 'to', 'dream', 'about', 'leaving', 'the', 'country', 'and', 'starting', 'new', 'life', 'he', 'escapes', 'from', 'prison', 'but', 'things', 'dont', 'go', 'as', 'planned']


## Remove Stopwords and Lemmatize

In [42]:
# Define functions for stopwords and lemmatization
def remove_stopwords(texts):
    return [[word for word in doc if word not in stop_words] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [43]:
nlp = stanza.Pipeline('en')

nlp("don't won't")

2022-12-04 16:14:17 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2022-12-04 16:14:17,830 : INFO : Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2022-12-04 16:14:19 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-12-04 16:14:19,106 : INFO : Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
|

[
  [
    {
      "id": 1,
      "text": "do",
      "lemma": "do",
      "upos": "AUX",
      "xpos": "VBP",
      "feats": "Mood=Imp|VerbForm=Fin",
      "head": 3,
      "deprel": "aux",
      "start_char": 0,
      "end_char": 2,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 2,
      "text": "n't",
      "lemma": "not",
      "upos": "PART",
      "xpos": "RB",
      "head": 3,
      "deprel": "advmod",
      "start_char": 2,
      "end_char": 5,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 3,
      "text": "wo",
      "lemma": "will",
      "upos": "AUX",
      "xpos": "MD",
      "feats": "VerbForm=Fin",
      "head": 0,
      "deprel": "root",
      "start_char": 6,
      "end_char": 8,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 4,
      "text": "n't",
      "lemma": "not",
      "upos": "PART",
      "xpos": "RB",
      "head": 3,
      "deprel": "advmod",
     

In [44]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
#!python3 -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 15.3 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [45]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

/opt/Anaconda/anaconda3/envs/ucsd1/lib/python3.8/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


['finnish', 'coal', 'miner', 'father', 'commit', 'suicide', 'frame', 'crime', 'commit', 'jail', 'start', 'dream', 'leave', 'country', 'start', 'new', 'life', 'escape', 'prison', 'thing', 'go', 'plan']


## Create the Dictionary and Corpus needed for Topic Modeling

In [46]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Frequency list   
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[0])

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1)]


In [47]:
id2word[10]

'jail'

In [48]:
[(id2word[cp[0]], cp[1])  for cp in corpus[10]]

[('life', 1),
 ('make', 2),
 ('take', 1),
 ('way', 1),
 ('event', 1),
 ('appear', 1),
 ('boy', 1),
 ('deeply', 1),
 ('destructive', 1),
 ('industrialist', 1),
 ('mean', 1),
 ('mother', 1),
 ('move', 1),
 ('newspaper', 1),
 ('reaction', 1),
 ('rest', 1),
 ('result', 1),
 ('rich', 1),
 ('self', 1),
 ('tyrannical', 1),
 ('ward', 1),
 ('well', 1),
 ('wound', 1)]

## Building the Topic Model
 - The coherence score is good for num_topics = 3

In [96]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [97]:
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.020*"take" + 0.020*"empire" + 0.010*"great" + 0.010*"friendly" + 0.010*"adventurous" + 0.010*"shark" + 0.010*"office" + 0.010*"home" + 0.010*"hungry" + 0.010*"clownfish"'), (1, '0.022*"life" + 0.022*"class" + 0.022*"city" + 0.012*"predict" + 0.012*"futuristic" + 0.012*"prophet" + 0.012*"work" + 0.012*"son" + 0.012*"fall" + 0.012*"planner"'), (2, '0.018*"love" + 0.018*"thing" + 0.018*"man" + 0.010*"yet" + 0.010*"life" + 0.010*"true" + 0.010*"event" + 0.010*"accomplish" + 0.010*"story" + 0.010*"far"'), (3, '0.014*"short" + 0.014*"story" + 0.014*"fallon" + 0.014*"pirate" + 0.008*"get" + 0.008*"action" + 0.008*"first" + 0.008*"bellhop" + 0.008*"transform" + 0.008*"hotel"'), (4, '0.002*"life" + 0.002*"suffer" + 0.002*"fate" + 0.002*"moment" + 0.002*"love" + 0.002*"operation" + 0.002*"learn" + 0.002*"cope" + 0.002*"number" + 0.002*"little"'), (5, '0.016*"mission" + 0.016*"endless" + 0.016*"memory" + 0.016*"commit" + 0.016*"exist" + 0.016*"summer" + 0.016*"start" + 0.009*"film" + 0.00

## Visualize the topic model

In [98]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)

/opt/Anaconda/anaconda3/envs/ucsd1/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [99]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.163982 -0.113896       1        1  20.568573
8     -0.147001 -0.098168       2        1  16.004110
3      0.027854  0.152323       3        1  13.963222
5     -0.032197  0.007899       4        1  11.589946
2     -0.009059  0.017339       5        1   9.254594
6     -0.007710  0.030316       6        1   8.342706
0     -0.025777 -0.014154       7        1   8.188456
1      0.012299  0.015028       8        1   6.186684
9      0.014661  0.007584       9        1   5.773595
4      0.002946 -0.004272      10        1   0.128115, topic_info=         Term      Freq     Total Category  logprob  loglift
97       film  5.000000  5.000000  Default  30.0000  30.0000
125      take  4.000000  4.000000  Default  29.0000  29.0000
23        man  3.000000  3.000000  Default  28.0000  28.0000
208  daughter  2.000000  2.000000  Default  27.0000  27.0000
340   mission  3.000000  3.000000  Default  26.0000  26.0000
..        ...       ...       ...      ...      ...      ...
240       end  0.001638  1.935050  Topic10  -6.2496  -0.4145
249      meet  0.001636  1.935145  Topic10  -6.2509  -0.4158
55        get  0.001635  2.567272  Topic10  -6.2514  -0.6989
199      love  0.001643  4.663324  Topic10  -6.2465  -1.2910
107      make  0.001640  3.938479  Topic10  -6.2486  -1.1241

[443 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
183       5  0.961895     ability
465       6  0.986472      accept
184       5  0.961895  accomplish
361       1  0.539926      action
361       3  0.539926      action
...     ...       ...         ...
133       2  0.585810         yet
133       5  0.585810         yet
182       1  0.671156       young
182       5  0.335578       young
182       7  0.335578       young

[347 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 9, 4, 6, 3, 7, 1, 2, 10, 5])

In [100]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.787064013074172


2022-12-04 17:49:10,270 : ERROR : worker encountered unexpected exception
Traceback (most recent call last):
  File "/opt/Anaconda/anaconda3/envs/ucsd1/lib/python3.8/site-packages/gensim/topic_coherence/text_analysis.py", line 561, in run
    self._run()
  File "/opt/Anaconda/anaconda3/envs/ucsd1/lib/python3.8/site-packages/gensim/topic_coherence/text_analysis.py", line 581, in _run
    self.accumulator.partial_accumulate(docs, self.window_size)
  File "/opt/Anaconda/anaconda3/envs/ucsd1/lib/python3.8/site-packages/gensim/topic_coherence/text_analysis.py", line 353, in partial_accumulate
    super(WordOccurrenceAccumulator, self).accumulate(texts, window_size)
  File "/opt/Anaconda/anaconda3/envs/ucsd1/lib/python3.8/site-packages/gensim/topic_coherence/text_analysis.py", line 296, in accumulate
    self.analyze_text(virtual_document, doc_num)
  File "/opt/Anaconda/anaconda3/envs/ucsd1/lib/python3.8/site-packages/gensim/topic_coherence/text_analysis.py", line 360, in analyze_text
    se


Coherence Score:  nan


/opt/Anaconda/anaconda3/envs/ucsd1/lib/python3.8/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
/opt/Anaconda/anaconda3/envs/ucsd1/lib/python3.8/site-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


# Keyword of topics

In [101]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.020*"take" + 0.020*"empire" + 0.010*"great" + 0.010*"friendly" + '
  '0.010*"adventurous" + 0.010*"shark" + 0.010*"office" + 0.010*"home" + '
  '0.010*"hungry" + 0.010*"clownfish"'),
 (1,
  '0.022*"life" + 0.022*"class" + 0.022*"city" + 0.012*"predict" + '
  '0.012*"futuristic" + 0.012*"prophet" + 0.012*"work" + 0.012*"son" + '
  '0.012*"fall" + 0.012*"planner"'),
 (2,
  '0.018*"love" + 0.018*"thing" + 0.018*"man" + 0.010*"yet" + 0.010*"life" + '
  '0.010*"true" + 0.010*"event" + 0.010*"accomplish" + 0.010*"story" + '
  '0.010*"far"'),
 (3,
  '0.014*"short" + 0.014*"story" + 0.014*"fallon" + 0.014*"pirate" + '
  '0.008*"get" + 0.008*"action" + 0.008*"first" + 0.008*"bellhop" + '
  '0.008*"transform" + 0.008*"hotel"'),
 (4,
  '0.002*"life" + 0.002*"suffer" + 0.002*"fate" + 0.002*"moment" + '
  '0.002*"love" + 0.002*"operation" + 0.002*"learn" + 0.002*"cope" + '
  '0.002*"number" + 0.002*"little"'),
 (5,
  '0.016*"mission" + 0.016*"endless" + 0.016*"memory" + 0.016*"commit" + '

In [102]:
lda_model[corpus][0]

([(5, 0.9828751)],
 [(0, [5]),
  (1, [5]),
  (2, [5]),
  (3, [5]),
  (4, [5]),
  (5, [5]),
  (6, [5]),
  (7, [5]),
  (8, [5]),
  (9, [5]),
  (10, [5]),
  (11, [5]),
  (12, [5]),
  (13, [5]),
  (14, [5]),
  (15, [5]),
  (16, [5]),
  (17, [5]),
  (18, [5]),
  (19, [5])],
 [(0, [(5, 0.9999759)]),
  (1, [(5, 1.9999806)]),
  (2, [(5, 0.9999759)]),
  (3, [(5, 0.9999758)]),
  (4, [(5, 0.9999759)]),
  (5, [(5, 0.9999759)]),
  (6, [(5, 0.9999759)]),
  (7, [(5, 0.9999758)]),
  (8, [(5, 0.9999758)]),
  (9, [(5, 0.9999759)]),
  (10, [(5, 0.9999759)]),
  (11, [(5, 0.9999759)]),
  (12, [(5, 0.9999761)]),
  (13, [(5, 0.9999759)]),
  (14, [(5, 0.99997646)]),
  (15, [(5, 0.9999759)]),
  (16, [(5, 0.9999759)]),
  (17, [(5, 1.9999804)]),
  (18, [(5, 0.9999759)]),
  (19, [(5, 0.9999759)])])

## Attach topics to movies DF

In [109]:
movies_filtered_df['topic'] = [lda_model[corpus][text][0][0][0] for text in range(len(movies_filtered_df['overview']))]

/tmp/ipykernel_76384/1793517637.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_filtered_df['topic'] = [lda_model[corpus][text][0][0][0] for text in range(len(movies_filtered_df['overview']))]


In [110]:
movies_filtered_df

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,...,spoken_languages,status,tagline,title,vote_average,vote_count,production_companies_number,production_countries_number,spoken_languages_number,topic
0,2,0,Drama|Crime,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,0.823904,Villealfa Filmproduction Oy,Finland,...,suomi,Released,NaN,Ariel,7.1,40,2,1,2,5
1,3,0,Drama|Comedy,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",0.47445,Villealfa Filmproduction Oy,Finland,...,English,Released,NaN,Shadows in Paradise,7.0,32,1,1,3,1
2,5,4000000,Crime|Comedy,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,1.698,Miramax Films,United States of America,...,English,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,485,2,1,1,3
3,6,0,Action|Thriller|Crime,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",1.32287,Universal Pictures,Japan,...,English,Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,6.5,69,3,2,1,3
4,8,42000,Documentary,tt0825671,en,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,0.054716,inLoops,Austria,...,English,Released,A Megacities remix.,Life in Loops (A Megacities RMX),6.4,4,1,1,5,8
5,9,0,Drama,tt0425473,de,Sonntag im August,NaN,0.001647,NaN,Germany,...,Deutsch,Released,NaN,Sunday in August,5.3,2,0,1,1,0
6,11,11000000,Adventure|Action|Science Fiction,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,10.492614,Lucasfilm,United States of America,...,English,Released,"A long time ago in a galaxy far, far away...",Star Wars,8.0,6168,2,1,1,0
7,12,94000000,Animation|Family,tt0266543,en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",9.915573,Pixar Animation Studios,United States of America,...,English,Released,"There are 3.7 trillion fish in the ocean, they...",Finding Nemo,7.6,5531,1,1,1,0
8,13,55000000,Comedy|Drama|Romance,tt0109830,en,Forrest Gump,A man with a low IQ has accomplished great thi...,10.351236,Paramount Pictures,United States of America,...,English,Released,"The world will never be the same, once you've ...",Forrest Gump,8.2,7204,1,1,1,2
9,14,15000000,Drama,tt0169547,en,American Beauty,"Lester Burnham, a depressed suburban father in...",8.191009,DreamWorks SKG,United States of America,...,English,Released,Look closer.,American Beauty,7.9,2994,2,1,1,1


/opt/Anaconda/anaconda3/envs/ucsd1/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/Anaconda/anaconda3/envs/ucsd1/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/Anaconda/anaconda3/envs/ucsd1/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/Anaconda/anaconda3/envs/ucsd1/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/Anaconda/anaconda3/envs/ucsd1/lib/pytho